# TESS Early Release TICA products custom extraction
## For Sector 36 Orbit 1& 2

April 9, 2021

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd
import lightkurve as lk
from astropy.time import Time
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## TESS Lightcurves

In [ ]:
btjd_now = Time.now().jd - 2457000.0
btjd_now = 2314.3505296814255

## Early Release data products

TICA now offers [early release data products](https://heasarc.gsfc.nasa.gov/docs/tess/a-beta-trial-of-quick-look-tess-orbit-data-early-release-of-sector-35-orbit-1-calibrated-full-frame-images.html) through TICA full frame images.

There are two ways to try to access TICA:

- With tess-cloud, vis-a-vis the demo on [this GitHub Issue](https://github.com/lightkurve/lightkurve/issues/993)
- Download the full FFI frames yourself from the [TICA curl scripts](https://archive.stsci.edu/hlsp/tica)

Both of these strategies are not well-proven!  Let's use `tess-point` to get the coordinates.

### Luhman 16

`$ python -m tess_stars2px -c 162.303282427 -53.317573814`

```bash
# TIC     |   RA      |   Dec     | EclipticLong | EclipticLat | Sector | Camera | Ccd | ColPix | RowPix | EdgeWarn
000000000 | 162.303282 | -53.317574 | 194.986936 | -53.902482 | 10 | 2 | 1 | 1766.327733 |    7.550842 | 0
000000000 | 162.303282 | -53.317574 | 194.986936 | -53.902482 | 36 | 3 | 1 |  541.096313 |  156.639815 | 0
000000000 | 162.303282 | -53.317574 | 194.986936 | -53.902482 | 37 | 3 | 2 | 1054.536935 |   55.021891 | 0
```

In [ ]:
from astropy.io import fits
import glob
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u

import numpy as np
import tqdm

In [ ]:
#object_name = 'Luhman 16'
#sr = lk.search_tesscut(object_name)
#tpf = sr.download(cutout_size=(41, 15))

Ok, so we want camera 4, ccd 1.

In [ ]:
tica_fns = glob.glob('../../data/TESS/TICA/s0036/cam3-ccd1/*.fits')

In [ ]:
len(tica_fns)

Get a preliminary image to set the mask

In [ ]:
hdus = fits.open(tica_fns[123])

hdr = hdus[0].header

In [ ]:
time_btjd = hdr['MIDTJD']

In [ ]:
wcs = WCS(hdr)

In [ ]:
coord = SkyCoord(162.303282427*u.deg, -53.317573814*u.deg)

yc, xc = wcs.world_to_pixel(coord)
xc, yc = int(xc), int(yc)
yc, xc

Wow, really close to what TESS-point said!  Now stack all the data into a datacube:

In [ ]:
dw = 17

In [ ]:
times = []
data = []

In [ ]:
import time

In [ ]:
time.sleep(10) # wait for the most recent file to complete downloading.

In [ ]:
for i, fn in tqdm.tqdm(enumerate(tica_fns)):
    hdus = fits.open(fn)
    hdr = hdus[0].header
    times.append(hdr['MIDTJD'])
    
    img = hdus[0].data
    subimg = img[xc-dw:xc+dw, yc-dw:yc+dw]
    data.append(subimg)
    hdus.close()

In [ ]:
datacube = np.array(data)
times = np.array(times)
indices = times.argsort()
times = times[indices]
datacube = datacube[indices, :, :]

In [ ]:
median_frame = np.nanmedian(datacube, axis=0)

In [ ]:
lo, mediumlo, mid, hi = np.nanpercentile(median_frame, (1, 15, 50, 90))

Ok, good spot-check, we are pointed at the right thing!

In [ ]:
vec = np.arange(34)
xx, yy = np.meshgrid(vec, vec)
dist = np.hypot(xx-17, yy-17)

In [ ]:
aperture_mask = (median_frame > mid) & (dist < 2)

In [ ]:
aperture_mask.sum()

In [ ]:
bkg_mask = (median_frame < mediumlo) & (dist > 15)

In [ ]:
bkg_mask.sum()

In [ ]:
plt.imshow(median_frame*~aperture_mask, vmin=mediumlo, vmax=hi);
plt.scatter([17], [17], marker='o', s=500, fc='none', ec='w')

Hmmm, where is the source?

In [ ]:
plt.imshow(aperture_mask);
plt.scatter([17], [17], marker='o', s=500, fc='none', ec='w')

In [ ]:
plt.imshow(bkg_mask);
plt.scatter([17], [17], marker='o', s=500, fc='none', ec='w')

In [ ]:
n_ap = np.sum(aperture_mask)

In [ ]:
n_bkg = np.sum(bkg_mask)

In [ ]:
flux = np.sum(datacube*aperture_mask, axis=(1,2)) - np.sum(datacube*bkg_mask, axis=(1,2))/n_bkg*n_ap

In [ ]:
from astropy.time import Time

In [ ]:
lc_tica = lk.LightCurve(time=Time(times,format='btjd'), flux=flux).normalize().remove_outliers()

Neat! We have a lightcurve.  Let's overplot the four IGRINS 2021A DDT epochs.

In [ ]:
df = pd.read_csv('../../data/IGRINS/2021A_metadata_log.csv')

In [ ]:
bad_mask = (((lc_tica.time.value > 2280.8) & (lc_tica.time.value<2282.3) ) | 
            ((lc_tica.time.value > 2293.8) & (lc_tica.time.value<2295.1) ) )

In [ ]:
lc_tica = lc_tica[~bad_mask]

In [ ]:
ax = lc_tica.plot(drawstyle='steps-mid')

for i in range(len(df)):
    ax.axvline(df.iloc[i].BTJD)
    
ax.axvline(-999, color=None, label='IGRINS Visits: March 11, 12, 13, & 17')
ax.fill_between([2280.8, 2282.3], 1.1, alpha=0.8, color='#ecf0f1', zorder=10, label='TESS thermal settling')
ax.fill_between([2293.8, 2295.1], 1.1, alpha=0.8, color='#ecf0f1', zorder=10)
ax.axvline(btjd_now, color='#e67e22', label='April 9, 2021', linestyle='dashed')
ax.set_ylim(0.93, 1.08)
ax.set_xlim(2280, 2315)
plt.legend(loc='best')
plt.savefig('../../figures/TESS_S36_LUH16_IGRINS_overview.png', bbox_inches='tight', dpi=300)

In [ ]:
#! xdg-open ../../figures/

In [ ]:
ax = lc_tica.plot(drawstyle='steps-mid', zorder=10, lw=1, label='TESS Sector 36 (Early Release)')

for i in range(len(df)):
    ax.axvline(df.iloc[i].BTJD, lw=0.3)
    
ax.axvline(-999, lw=0.3, label='IGRINS visits')
ax.set_xlim(2285, 2292)
plt.legend(loc='best')

plt.savefig('../../figures/TESS_S36_LUH16_IGRINS_DDT.png', bbox_inches='tight', dpi=300)

Neat! Let's save the lightcurve.

In [ ]:
hdu = lc_tica.to_fits('../../data/TESS/LUH16_TESS_S36_lk_custom_4pixel.fits', overwrite=True)

In [ ]:
out = lc_tica.to_csv('../../data/TESS/LUH16_TESS_S36_lk_custom_4pixel.csv')

In [ ]:
df['TESS_flux'] = 0.0

In [ ]:
for i in range(8):
    df.loc[i, "TESS_flux"] = lc_tica[np.argmin(np.abs(lc_tica.time.value - df.iloc[i].BTJD))]['flux'].value

In [ ]:
df

In [ ]:
plt.hist(lc_tica.flux.value, bins=np.arange(0.92, 1.1, 0.002));
for i in range(8):
    plt.axvline(df.TESS_flux[i], color='#e67e22', linestyle='dashed')
plt.xlabel('TESS Flux')
plt.title('TESS Sector 36')

In [ ]:
lc_s10 = lk.TessLightCurve.read('../../data/TESS/lightkurve_custom_4pixel.fits')

In [ ]:
plt.hist(lc_s10.flux.value, bins=np.arange(0.92, 1.1, 0.002));
for i in range(8):
    plt.axvline(df.TESS_flux[i], color='#e67e22', linestyle='dashed')

In [ ]:
lc_s10.plot()